# Web Scraping for Real State
### Personal Code: José Enrique Mejía Gamarra

- [1. NODE IDENTIFICATION](#1)
- [2. LOAD REQUIRED LIBRARIES](#2)
- [3. CONFIGURATION](#3)
- [4. OBTAIN THE DATA](#4)
- [5. VIEW DATAFRAME](#5)
- [6. SEND THE INFORMATION TO PostgreSQL](#6)

## The page from where information will be obtained is:
## https://urbania.pe/buscar/alquiler-de-departamentos

<a name='1'></a>
## 1. NODE IDENTIFICATION
### Nodes are identified that contain information regarding: "nombre del anuncio", "m2" and "precio de alquiler"

### Node for "nombre de anuncio"
[@class='list-card-container ']/div/div/div[2]/div[1]/div/div[1]/div[1]/h2/a

### Node for "m2"
[@class='list-card-container ']/div/div/div[2]/div[1]/div/ul/li[1]

### Node for "precio de alquiler"
[@class='list-card-container ']/div/div/div[1]/div[2]/div[1]/div/span/b

<a name='2'></a>
## 2. LOAD REQUIRED LIBRARIES

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

<a name='3'></a>
## 3. CONFIGURATION

In [2]:
# Define the Firefox path
binary = FirefoxBinary(r'C:\\Program Files\\Mozilla Firefox\\firefox.exe')

# Create a profile
profile = webdriver.FirefoxProfile()

# Run the browser
driver = webdriver.Firefox(firefox_profile=profile,
                           firefox_binary=binary, 
                           executable_path='C:\\Users\\usuario\\Personal_Notebook\\geckodriver.exe')

In [3]:
nro_pag=5         #Number of pages to be analyzed
n=0               #The number of pages starts with 0
totalx=0          #The total number of ads.

In [4]:
#Define the dataframe where the data will be stored.
data_all = {'Anuncio':[],             #First create a dictionary
             'Area':[],
             'Precio':[]}
df_data_all = pd.DataFrame(data_all)  #Convert the dictionary to DataFrame

<a name='4'></a>
## 4. OBTAIN THE DATA

In [5]:
Flag=True #The condition value.
while Flag:
    
    #Update the value of n for each iteration
    n=n+1  
    
    #Define the url
    url = "https://urbania.pe/buscar/alquiler-de-departamentos?page="+str(n) 
   
    #We open the url page we specified.
    driver.get(url)
    
    #Wait 5 seconds for the page to load.
    time.sleep(10) 
    
    #Nombre de anuncio (Name of the ad)
    div_nombres = driver.find_elements(By.XPATH, "//*[@class='list-card-container ']/div/div/div[2]/div[1]/div/div[1]/div[1]/h2/a")
    list_new_nombres = list(map(lambda x: x.text, div_nombres))
    
    #Area (Area)
    div_area = driver.find_elements(By.XPATH, "//*[@class='list-card-container ']/div/div/div[2]/div[1]/div/ul/li[1]")
    list_new_area = list(map(lambda x: x.text, div_area))
    
    #Precio (Price)
    div_precio = driver.find_elements(By.XPATH, "//*[@class='list-card-container ']/div/div/div[1]/div[2]/div[1]/div/span/b")
    list_new_precio = list(map(lambda x: x.text, div_precio))
    
    #Calculate number of ads
    number_list = len(list_new_nombres)
    totalx=totalx+number_list
    
    #Generate a dataframe
    data_pag = {'Anuncio':list_new_nombres,
                 'Area':list_new_area,
                 'Precio':list_new_precio}
    df_data_pag = pd.DataFrame(data_pag)
    
    #Concatenate
    df_data_all=pd.concat([df_data_all, df_data_pag], axis=0)

    print("The page "+str(n)+"° contain "+str(number_list)+" ads, the total so far is "+str(totalx))
    
    if n==nro_pag:
        Flag=False

La pagina 1° contiene 25 anuncios, el total hasta ahora es 25
La pagina 2° contiene 25 anuncios, el total hasta ahora es 50
La pagina 3° contiene 25 anuncios, el total hasta ahora es 75
La pagina 4° contiene 25 anuncios, el total hasta ahora es 100
La pagina 5° contiene 25 anuncios, el total hasta ahora es 125


<a name='5'></a>
## 5. VIEW DATAFRAME

In [6]:
df_books_all=df_books_all.reset_index(drop=True)
df_books_all

,Anuncio,Area,Precio
0,Magno - Calle Andalucía 250 Miraflores,37 unidades,Consultar precio
1,Alquilo Hermoso Departamento en San Isidro!,60 m² totales,"S/ 1,963"
2,Frente Al Mar - Dpto Amoblado y Equipado Malec...,60 m² totales,"S/ 2,450"
3,Alquiler Departamento Penthause en Miraflores,340 m² totales,"S/ 8,640"
4,Alquilo Departamento de Estreno en La Mejor Zo...,100 m² totales,"S/ 3,200"
...,...,...,...
120,Gran Alquiler de Departamento en Barranco,200 m² totales,"S/ 6,707"
121,Alquiler Departamento Amoblado Ubicado en Mira...,65 m² totales,"S/ 2,160"
122,Alquiler de Duplex en Miraflores con Vista Pan...,120 m² totales,"S/ 3,500"
123,Amoblado y Equipado,106 m² totales,"S/ 3,916"


#### As can be seen, the first value is actually a pre-sale of apartments, so it will have to be eliminated.

In [7]:
df_books=df_books_all.drop([0],axis=0).reset_index(drop=True)
df_books

,Anuncio,Area,Precio
0,Alquilo Hermoso Departamento en San Isidro!,60 m² totales,"S/ 1,963"
1,Frente Al Mar - Dpto Amoblado y Equipado Malec...,60 m² totales,"S/ 2,450"
2,Alquiler Departamento Penthause en Miraflores,340 m² totales,"S/ 8,640"
3,Alquilo Departamento de Estreno en La Mejor Zo...,100 m² totales,"S/ 3,200"
4,"Alquiler de Departamento Amoblado y Equipado, ...",67 m² totales,"S/ 3,500"
...,...,...,...
119,Gran Alquiler de Departamento en Barranco,200 m² totales,"S/ 6,707"
120,Alquiler Departamento Amoblado Ubicado en Mira...,65 m² totales,"S/ 2,160"
121,Alquiler de Duplex en Miraflores con Vista Pan...,120 m² totales,"S/ 3,500"
122,Amoblado y Equipado,106 m² totales,"S/ 3,916"


<a name='6'></a>
## 6. SEND THE INFORMATION TO PostgreSQL

In [8]:
#The necessary libraries are installed
from sqlalchemy import create_engine
import pandas as pd
import psycopg2 
import io

In [9]:
#Credentials
Usuario='postgres'
Pass='*******'        #For security reasons, this information will not be displayed.
IP='69.164.192.245'
Puerto='5432'
Base_datos='web_scraping'

In [10]:
#Connection to DataBase
engine = create_engine('postgresql+psycopg2://'+Usuario+':'+Pass+'@'+IP+':'+Puerto+'/'+Base_datos)
df_data_all.to_sql('jose_mejia', engine)